In [42]:
# Section 1: Load Environment Variables
import os
from dotenv import load_dotenv

# Load the environment variables from db.env
load_dotenv('db.env')

# Check if environment variables are loaded correctly
print("Driver:", os.getenv('DB_DRIVER'))
print("Host:", os.getenv('DB_HOST'))
# print("User:", os.getenv('DB_USERNAME'))
print("Database:", os.getenv('DB_DATABASE'))
print("Port:", os.getenv('DB_PORT'))
# print("Password:", os.getenv('DB_PASSWORD'))


Driver: Terrapin New PostgreSQL
Host: 23.88.78.5
Database: tp_data
Port: 5432


In [43]:
# Section 2: Check .ini File Contents
import configparser

config = configparser.ConfigParser()
config.read(os.path.expanduser('~/.odbc.ini'))

print("Sections:", config.sections())

for section in config.sections():
    print(f"Named: {section}")
    for key, value in config.items(section):
        print(f"{key}: ") #{value}") # print(f"{key}: {value}")

Sections: ['Terrapin New PostgreSQL']
Named: Terrapin New PostgreSQL
description: 
driver: 
server: 
database: 
user: 
password: 
port: 


In [52]:
# Section 3: Check Database Connection
from sqlalchemy import create_engine, text

# Construct the connection string
connection_string = (
    f"postgresql+psycopg2://{os.getenv('DB_USERNAME')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_DATABASE')}"
)
# print("Connection String:", connection_string)  # Debug print  !!! really useful but not to be disclosed

# Create the engine and test the connection
try:
    engine = create_engine(connection_string)
    with engine.connect() as connection:
        result = connection.execute(text("SELECT version();"))
        row = result.fetchone()
        print(row)
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")


('PostgreSQL 16.3 (Ubuntu 16.3-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)
Connection successful


In [51]:
# Section 4: Fetch Data in Expected Format
import pyodbc

try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM trades LIMIT 5;")  # Replace with actual desired table name
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    conn.close()
    print("Data fetched successfully")
except Exception as e:
    print(f"Data fetch failed: {e}")


Data fetch failed: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found, and no default driver specified (0) (SQLDriverConnect)')


In [46]:
# Section 5: Fetch Data for ISINs
import pandas as pd
from sqlalchemy import create_engine

def fetch_yield_curve_data(isin, engine):
    query = f"""
        SELECT isin, duration, last_trade_date, price, yield_to_maturity
        FROM bonds_pricing
        WHERE isin IN (
            SELECT isin
            FROM bonds_reference
            WHERE lei = (
                SELECT lei
                FROM bonds_reference
                WHERE isin = '{isin}'
            )
        )
        AND price is not null
        AND duration is not null
        ORDER BY duration
    """
    with engine.connect() as connection:
        df = pd.read_sql_query(query, connection)
    return df

try:
    connection_string = (
        f"postgresql+psycopg2://{os.getenv('DB_USERNAME')}:{os.getenv('DB_PASSWORD')}@"
        f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_DATABASE')}"
    )
    engine = create_engine(connection_string)
    
    # ISINs for German and Italian 10-year government bonds
    german_isin = 'DE0001102572'
    italian_isin = 'IT0005437147'
    
    # Fetch data for German bond
    german_df = fetch_yield_curve_data(german_isin, engine)
    print("German 10-year Government Bond Data:")
    print(german_df)
    
    # Fetch data for Italian bond
    italian_df = fetch_yield_curve_data(italian_isin, engine)
    print("\nItalian 10-year Government Bond Data:")
    print(italian_df)
    
except Exception as e:
    print(f"Data fetch failed: {e}")


German 10-year Government Bond Data:
            isin  duration last_trade_date     price  yield_to_maturity
0   DE000BU0E071   0.01369      2024-07-12  99.99061            0.68833
1   DE0001102366   0.08214      2024-07-16  99.79000            3.56978
2   DE000BU0E089   0.10130      2024-07-15  99.67200            3.29638
3   DE0001104891   0.16153      2024-07-16  99.53500            3.32024
4   DE000BU0E097   0.17522      2024-07-16  99.41000            3.43479
..           ...       ...             ...       ...                ...
80  DE0001102614  21.76527      2024-07-16  83.62000            2.61069
81  DE0001030757  21.77209      2024-07-16  83.74800            2.60362
82  DE0001102481  26.08074      2024-07-16  51.42200            2.58297
83  DE0001030724  26.08076      2024-07-16  51.60900            2.56869
84  DE0001102572  28.08212      2024-07-16  49.12300            2.56361

[85 rows x 5 columns]

Italian 10-year Government Bond Data:
             isin  duration last_trad

In [36]:
# Section 6 And now plotting in a chart

import plotly.express as px

# Plotting the data using Plotly
if not german_df.empty and not italian_df.empty:
    fig1 = px.scatter(german_df, x="duration", y="yield_to_maturity", color="isin",
                    title="Yield Curve Comparison for German and Italian Bonds",
                    labels={"duration": "Duration", "yield_to_maturity": "Yield to Maturity"})
        
    fig1.add_scatter(x=italian_df["duration"], y=italian_df["yield_to_maturity"], mode="markers", name="Italian Bonds",
                        marker=dict(color='red', size=10))

    fig1.update_traces(marker=dict(size=12),
                        selector=dict(mode='markers', name="Italian Bonds"))

    fig1.show()

    fig2 = px.scatter(german_df, x="duration", y="price", color="isin",
                    title="Price Comparison for German and Italian Bonds",
                    labels={"duration": "Duration", "price": "Bond price"})
        
    fig2.add_scatter(x=italian_df["duration"], y=italian_df["price"], mode="markers", name="Italian Bonds",
                        marker=dict(color='red', size=10))

    fig2.update_traces(marker=dict(size=12),
                        selector=dict(mode='markers', name="Italian Bonds"))
    
    fig2.show()
    